In [26]:
from sympy import *
import numpy as np
from scipy.spatial.transform import Rotation

In [27]:
# -- States --
px, py, pz = symbols(r"p_x, p_y, p_z")
position = Matrix([px, py, pz])
vx, vy, vz = symbols(r"v_x, v_y, v_z")
velocity = Matrix([vx, vy, vz])

qw, qx, qy, qz = symbols(r"q_w, q_x, q_y, q_z")
quaternion = Matrix([qw, qx, qy, qz])

states_s = Matrix([position, velocity])

In [28]:
# -- Inputs --
ax, ay, az = symbols(r"a_x, a_y, a_z")
accelerometer_reading = Matrix([ax, ay, az])

inputs_u = [accelerometer_reading]

In [29]:
# -- Parameters --
dt = Symbol(r"\Delta t") # Timestep, s

g = symbols("g")
gravity_vec = Matrix([0, 0, g]) # Acceleration due to gravity in world frame, 3x m/s^2

pressure_sea_level = symbols(r"P_0") # Sea level atmospheric pressure [Pa]
universal_gas_constant = symbols(r"R") # Universal gas constant [J/(mol*K)]
air_molar_mass = symbols(r"M") # Molar mass of dry air [kg/mol]
room_temp = symbols(r"T_r") # Room temperature [K]

In [30]:
def rot_mat_from_quat(q):
    rotation_matrix = Matrix([
        [1 - 2*q[2]**2 - 2*q[3]**2, 2*q[1]*q[2] - 2*q[3]*q[0], 2*q[1]*q[3] + 2*q[2]*q[0]],
        [2*q[1]*q[2] + 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2, 2*q[2]*q[3] - 2*q[1]*q[0]],
        [2*q[1]*q[3] - 2*q[2]*q[0], 2*q[2]*q[3] + 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]
    ])
    return rotation_matrix

The state transition function ($f$) is defined as follows:
$$
s_{k+1} = f(s_k, u_k)
$$

In [31]:
accel_world_frame = (
    rot_mat_from_quat(quaternion) * (accelerometer_reading) + gravity_vec
)

state_transition_function_f = Matrix([
    position + velocity * dt + 1/2 * accel_world_frame * dt**2, # = world frame p_next
    velocity + accel_world_frame * dt, # = world frame v_next
])
print(state_transition_function_f)

Matrix([[\Delta t**2*(0.5*a_x*(-2*q_y**2 - 2*q_z**2 + 1) + 0.5*a_y*(-2*q_w*q_z + 2*q_x*q_y) + 0.5*a_z*(2*q_w*q_y + 2*q_x*q_z)) + \Delta t*v_x + p_x], [\Delta t**2*(0.5*a_x*(2*q_w*q_z + 2*q_x*q_y) + 0.5*a_y*(-2*q_x**2 - 2*q_z**2 + 1) + 0.5*a_z*(-2*q_w*q_x + 2*q_y*q_z)) + \Delta t*v_y + p_y], [\Delta t**2*(0.5*a_x*(-2*q_w*q_y + 2*q_x*q_z) + 0.5*a_y*(2*q_w*q_x + 2*q_y*q_z) + 0.5*a_z*(-2*q_x**2 - 2*q_y**2 + 1) + 0.5*g) + \Delta t*v_z + p_z], [\Delta t*(a_x*(-2*q_y**2 - 2*q_z**2 + 1) + a_y*(-2*q_w*q_z + 2*q_x*q_y) + a_z*(2*q_w*q_y + 2*q_x*q_z)) + v_x], [\Delta t*(a_x*(2*q_w*q_z + 2*q_x*q_y) + a_y*(-2*q_x**2 - 2*q_z**2 + 1) + a_z*(-2*q_w*q_x + 2*q_y*q_z)) + v_y], [\Delta t*(a_x*(-2*q_w*q_y + 2*q_x*q_z) + a_y*(2*q_w*q_x + 2*q_y*q_z) + a_z*(-2*q_x**2 - 2*q_y**2 + 1) + g) + v_z]])


In [32]:
state_transition_function_f

Matrix([
[        \Delta t**2*(0.5*a_x*(-2*q_y**2 - 2*q_z**2 + 1) + 0.5*a_y*(-2*q_w*q_z + 2*q_x*q_y) + 0.5*a_z*(2*q_w*q_y + 2*q_x*q_z)) + \Delta t*v_x + p_x],
[        \Delta t**2*(0.5*a_x*(2*q_w*q_z + 2*q_x*q_y) + 0.5*a_y*(-2*q_x**2 - 2*q_z**2 + 1) + 0.5*a_z*(-2*q_w*q_x + 2*q_y*q_z)) + \Delta t*v_y + p_y],
[\Delta t**2*(0.5*a_x*(-2*q_w*q_y + 2*q_x*q_z) + 0.5*a_y*(2*q_w*q_x + 2*q_y*q_z) + 0.5*a_z*(-2*q_x**2 - 2*q_y**2 + 1) + 0.5*g) + \Delta t*v_z + p_z],
[                                      \Delta t*(a_x*(-2*q_y**2 - 2*q_z**2 + 1) + a_y*(-2*q_w*q_z + 2*q_x*q_y) + a_z*(2*q_w*q_y + 2*q_x*q_z)) + v_x],
[                                      \Delta t*(a_x*(2*q_w*q_z + 2*q_x*q_y) + a_y*(-2*q_x**2 - 2*q_z**2 + 1) + a_z*(-2*q_w*q_x + 2*q_y*q_z)) + v_y],
[                                  \Delta t*(a_x*(-2*q_w*q_y + 2*q_x*q_z) + a_y*(2*q_w*q_x + 2*q_y*q_z) + a_z*(-2*q_x**2 - 2*q_y**2 + 1) + g) + v_z]])

$$
s_{k+1} = As_k + Bu_k
$$

In [33]:
state_transition_matrix_A = state_transition_function_f.jacobian(states_s)
print(state_transition_matrix_A)

Matrix([[1, 0, 0, \Delta t, 0, 0], [0, 1, 0, 0, \Delta t, 0], [0, 0, 1, 0, 0, \Delta t], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])


In [34]:
state_transition_matrix_A

Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]])

In [35]:
input_matrix_B = state_transition_function_f.jacobian(inputs_u)

In [36]:
input_matrix_B

Matrix([
[\Delta t**2*(-1.0*q_y**2 - 1.0*q_z**2 + 0.5),     \Delta t**2*(-1.0*q_w*q_z + 1.0*q_x*q_y),      \Delta t**2*(1.0*q_w*q_y + 1.0*q_x*q_z)],
[     \Delta t**2*(1.0*q_w*q_z + 1.0*q_x*q_y), \Delta t**2*(-1.0*q_x**2 - 1.0*q_z**2 + 0.5),     \Delta t**2*(-1.0*q_w*q_x + 1.0*q_y*q_z)],
[    \Delta t**2*(-1.0*q_w*q_y + 1.0*q_x*q_z),      \Delta t**2*(1.0*q_w*q_x + 1.0*q_y*q_z), \Delta t**2*(-1.0*q_x**2 - 1.0*q_y**2 + 0.5)],
[         \Delta t*(-2*q_y**2 - 2*q_z**2 + 1),            \Delta t*(-2*q_w*q_z + 2*q_x*q_y),             \Delta t*(2*q_w*q_y + 2*q_x*q_z)],
[            \Delta t*(2*q_w*q_z + 2*q_x*q_y),          \Delta t*(-2*q_x**2 - 2*q_z**2 + 1),            \Delta t*(-2*q_w*q_x + 2*q_y*q_z)],
[           \Delta t*(-2*q_w*q_y + 2*q_x*q_z),             \Delta t*(2*q_w*q_x + 2*q_y*q_z),          \Delta t*(-2*q_x**2 - 2*q_y**2 + 1)]])

The observation model ($h$) is defined a follows:
$$
y_k = h(s_k, u_k)
$$

In [37]:
def alt_temp_to_pressure(altitude_m):
    # Calculate pressure using barometric formula
    pressure = pressure_sea_level * exp(
        air_molar_mass * g * altitude_m / (room_temp * universal_gas_constant)
    )
    return pressure

In [38]:
observation_model_h = Matrix([
    alt_temp_to_pressure(pz), # barometer
])

In [49]:
observation_model_h

Matrix([[P_0*exp(M*g*p_z/(R*T_r))]])

In [40]:
observation_matrix_H = observation_model_h.jacobian(states_s)
print(observation_matrix_H)

Matrix([[0, 0, M*P_0*g*exp(M*g*p_z/(R*T_r))/(R*T_r), 0, 0, 0]])


In [41]:
observation_matrix_H

Matrix([[0, 0, M*P_0*g*exp(M*g*p_z/(R*T_r))/(R*T_r), 0, 0, 0]])

In [42]:
# Substitute equilibrium point values:
# Equilibrium quaternion
# Zero biases
subs_dict = {
    px: 0, py: 0, pz: 0.1,
    vx: 0, vy: 0, vz: 0, 
    qw: 0.8375417, qx: 0.2202814, qy: 0.3213938, qz: 0.3830222,
    dt: 0.1,
    g: 9.81,
    pressure_sea_level: 101325, # Pa
    room_temp: 295.15, # K
    universal_gas_constant: 8.31447, # J/(mol*K)
    air_molar_mass: 0.0289644, # kg/mol
}

A_num = state_transition_matrix_A.subs(subs_dict).evalf()
B_num = input_matrix_B.subs(subs_dict).evalf()
H_num = observation_matrix_H.subs(subs_dict).evalf()

# Convert sympy matrices to numpy arrays for observability calculation
A_np = np.array(A_num.tolist(), dtype=np.float64)
H_np = np.array(H_num.tolist(), dtype=np.float64)

In [43]:
A_num

Matrix([
[1.0,   0,   0, 0.1,   0,   0],
[  0, 1.0,   0,   0, 0.1,   0],
[  0,   0, 1.0,   0,   0, 0.1],
[  0,   0,   0, 1.0,   0,   0],
[  0,   0,   0,   0, 1.0,   0],
[  0,   0,   0,   0,   0, 1.0]])

In [44]:
B_num

Matrix([
[ 0.0025000001962872, -0.0024999998831042,  0.0035355337606854],
[ 0.0039159414074106,   0.003047700991212, -0.0006139389789202],
[-0.0018480804317438,  0.0030759581857674,   0.003481821301356],
[  0.050000003925744,  -0.049999997662084,   0.070710675213708],
[  0.078318828148212,    0.06095401982424,  -0.012278779578404],
[ -0.036961608634876,   0.061519163715348,    0.06963642602712]])

In [45]:
H_num

Matrix([[0, 0, 11.7321559332405, 0, 0, 0]])

In [46]:
# Build observability matrix [H; H A; H A^2; ... H A^(n-1)]
n_states = A_np.shape[0]
obs_matrix = H_np.copy()
for i in range(1, n_states):
    obs_matrix = np.vstack((obs_matrix, H_np @ np.linalg.matrix_power(A_np, i)))

rank_obs = np.linalg.matrix_rank(obs_matrix)

print(f"\nObservability matrix rank: {rank_obs} out of {n_states} states")

if rank_obs == n_states:
    print("The system is fully observable at the equilibrium point.")
else:
    print("The system is NOT fully observable at the equilibrium point.")


Observability matrix rank: 2 out of 6 states
The system is NOT fully observable at the equilibrium point.


In [47]:
Matrix(obs_matrix.tolist())

Matrix([
[0, 0, 11.7321559332405, 0, 0,                0],
[0, 0, 11.7321559332405, 0, 0, 1.17321559332405],
[0, 0, 11.7321559332405, 0, 0,  2.3464311866481],
[0, 0, 11.7321559332405, 0, 0, 3.51964677997215],
[0, 0, 11.7321559332405, 0, 0,  4.6928623732962],
[0, 0, 11.7321559332405, 0, 0, 5.86607796662025]])

In [48]:
from scipy.linalg import null_space

nullsp = null_space(obs_matrix)
if nullsp.shape[1] > 0:
    print("Basis vectors for unobservable states:")
    for i in range(nullsp.shape[1]):
        print(np.round(nullsp[:, i]))
        unobserved_states = [state for state, include in zip(states_s, abs(np.round(nullsp[:, i])) > 0.1) if include]
        print(f"Which corresponds to: {unobserved_states}")

Basis vectors for unobservable states:
[0. 0. 0. 0. 1. 0.]
Which corresponds to: [v_y]
[ 0. -1.  0.  0.  0.  0.]
Which corresponds to: [p_y]
[0. 0. 0. 1. 0. 0.]
Which corresponds to: [v_x]
[1. 0. 0. 0. 0. 0.]
Which corresponds to: [p_x]
